In [3]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.nn.functional as F


from torch.utils import data
import random
import numpy as np
from itertools import product
import argparse

from train_f import *
from Dataset import Dataset
from Models import *
from args import args
import os


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [1]:
#args
mini = 0
medium = 0
medium1 = 0
lr = 0.001
model_idx = 2
epochs = 1
batch_size = 20
loss_weight = 80
weight_decay = 0.0
print_freq = 400
target_cat = 'count'
target_class = 0
plot_label = ''
load_model = 0
save_name = ''
conv1_out, conv3_out, conv5_out = 6,8,10
record_results = 0
#yfloss_weight = torch.Tensor([1]).to(device)
vel = 0
normalize = 0
C_model = ''

In [2]:
# the following four variables are global variables that record the statistics 
# for each epoch so that the plot can be produced
TRAIN_LOSS,VAL_LOSS, VAL_ACC, VAL_RECALL, VAL_PRECISION = [],[],[],[],[]
BEST_VAL_LOSS = 999999999
BEST_RECALL = 0
BEST_PRECISION = 0
BEST_F1SCORE = 0
BEST_ACC = 0
EPSILON = 1e-5
if not os.path.exists('pretrained'):
    os.makedirs('pretrained')

NameError: name 'os' is not defined

In [4]:
def initial_loss(train_loader, val_loader, model, criterion, target_class):
    #AverageMeter is a object that record the sum, avg, count and val of the target stats
    train_losses = AverageMeter()
    val_losses = AverageMeter()  
    correct = 0
    # ptotal = 0  #count of all positive predictions
    # tp = 0    #true positive
    total = 0 #total count of data
    TPRs = AverageMeter()
    FPRs = AverageMeter()
    # switch to train mode
    model.eval()
    
    with torch.no_grad():
        for i, (input, target) in enumerate(train_loader):
            input = input.to(device).float()
            if target_class == 0:
                target = target.to(device).long()
            elif target_class == 1:
                target = target.to(device).float()
            # compute output
            output = model(input)
            # print("target1: ", target.size())
            # print("output: ", output.size())
            loss = criterion(output, target)
            # measure accuracy and record loss
            train_losses.update(loss.item(), input.size(0))

        for i, (input, target) in enumerate(val_loader):
            # add a dimension, from (1, 32, 32, 32) to (1,1,32,32,32)

            input = input.to(device).float()
            if target_class == 0:
                target = target.to(device).long()
            elif target_class == 1:
                target = target.to(device).float()
            # compute output
            output = model(input)
            loss = criterion(output, target)
            # measure accuracy and record loss
            val_losses.update(loss.item(), input.size(0))
            if target_class == 0:
                outputs = F.softmax(output, dim=1)
                predicted = outputs.max(1)[1]
                total += np.prod(target.shape)
                correct += predicted.eq(target.view_as(predicted)).sum().item()
                #ptotal += (target.view_as(predicted) >= 1).sum().item()
                #tp += torch.mul(predicted.eq(target.view_as(predicted)),(target.view_as(predicted)>= 1)).sum().item()
                TPR, gp, FPR, gf = confusion_matrix_calc(predicted,target)
                TPRs.update(TPR,gp)
                FPRs.update(FPR,gf)            
            loss = criterion(output, target)
            # measure accuracy and record loss
            val_losses.update(loss.item(), input.size(0))  

    # recall =  tp/ptotal*100  #recall = true positive / count of all positive predictions  
    if target_class == 0:
        acc = correct/total*100
        recall = TPRs.avg * 100
        precision = TPRs.sum/(TPRs.sum + FPRs.sum + EPSILON) * 100  
        VAL_RECALL.append(recall)
        VAL_ACC.append(acc)
        VAL_PRECISION.append(precision)  

    TRAIN_LOSS.append(train_losses.avg)
    VAL_LOSS.append(val_losses.avg)
    if target_class == 0:
        print('Epoch Train Loss {train_losses.avg:.4f}, Test Loss {val_losses.avg:.4f},\
         Test Accuracy {acc:.4f},  Test Recall {recall:.4f}\t Precision {precision:.4f}\t'.format(train_losses = train_losses, \
            val_losses=val_losses,acc=acc, recall = recall, precision = precision))
    else:
        print('Epoch Train Loss {train_losses.avg:.4f}, Test Loss {val_losses.avg:.4f}'\
            .format(train_losses = train_losses, val_losses=val_losses))

def train(train_loader, model, criterion, optimizer, epoch, print_freq, target_class):


    batch_time = AverageMeter()
    losses = AverageMeter()
    data_time = AverageMeter()


    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        # add a dimension, from (1, 32, 32, 32) to (1,1,32,32,32)
        input = input.to(device).float()
        if target_class == 0:
            target = target.to(device).long()
        elif target_class == 1:
            target = target.to(device).float()
        # compute output
        output = model(input)

        #print(torch.nonzero(target).size())
        loss = criterion(output, target)
        # measure accuracy and record loss
        losses.update(loss.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses))
    TRAIN_LOSS.append(losses.avg)
    print('Epoch {0} : Train: Loss {loss.avg:.4f}\t'.format(epoch, loss=losses))
    


def validate(val_loader, model, criterion, epoch, target_class, save_name):
    global BEST_VAL_LOSS
    global BEST_RECALL
    global BEST_PRECISION
    global BEST_F1SCORE
    global BEST_ACC
    batch_time = AverageMeter()
    val_losses = AverageMeter()
    TPRs = AverageMeter()
    FPRs = AverageMeter()  
    total = 0
    correct = 0
    model.eval()
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            input = input.to(device).float()
            if target_class == 0:
                target = target.to(device).long()
            elif target_class == 1:
                target = target.to(device).float()
            
            
            # compute output
            output = model(input)
            if target_class == 0:
                outputs = F.softmax(output, dim=1)
                predicted = outputs.max(1)[1]
                total += np.prod(target.shape)
                correct += predicted.eq(target.view_as(predicted)).sum().item()
                TPR, gp, FPR, gf = confusion_matrix_calc(predicted,target)
                TPRs.update(TPR,gp)
                FPRs.update(FPR,gf)
            loss = criterion(output, target)
            # measure accuracy and record loss
            val_losses.update(loss.item(), input.size(0))

    
    if target_class == 0:
        recall = TPRs.avg * 100
        precision = TPRs.sum/(TPRs.sum + FPRs.sum + EPSILON) * 100
        F1score = 2*((precision*recall)/(precision+recall+ EPSILON))
        acc = correct/total*100
        VAL_RECALL.append(recall)
        VAL_ACC.append(acc)
        VAL_PRECISION.append(precision)
        if val_losses.avg < BEST_VAL_LOSS:
            BEST_RECALL = recall
            BEST_PRECISION = precision
            BEST_F1SCORE = F1score
            BEST_ACC = acc
    
    if val_losses.avg < BEST_VAL_LOSS:
        if len(save_name) > 0:
            #torch.save(model, 'pretrained/' + str(save_name) + '.pt')
            torch.save(model.state_dict(), 'pretrained/' + str(save_name) + '.pth')
        BEST_VAL_LOSS = val_losses.avg
    VAL_LOSS.append(val_losses.avg)
    if target_class == 0:
        print('Epoch {0} :Test Loss {val_losses.avg:.4f},\
         Test Accuracy {acc:.4f},  Test Recall {recall:.4f}\t Precision {precision:.4f} F1 score  {F1score:.4f}\t'.format(epoch, \
            val_losses=val_losses,acc=acc, recall = recall, precision = precision, F1score = F1score))
    else:
        print('Epoch {0} : Test Loss {val_losses.avg:.4f}'\
            .format(epoch, val_losses=val_losses))

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#index for the cube, each tuple corresponds to a cude
#test data
if mini:
    train_data = [(832, 640, 224),(864, 640, 224)]
    val_data = [(832, 640, 224),(864, 640, 224)]
    test_data = [(832, 640, 224),(864, 640, 224)]
else:
    if medium1:
        data_range = 130
        random_idx = 1
    elif medium:
        data_range = 512
        random_idx = 1
    else:
        data_range = 1024
        random_idx = 0

    pos=list(np.arange(0,data_range,32))
    ranges=list(product(pos,repeat=3))
    random.seed(7)
    if random_idx == 1:
        random.shuffle(ranges)
        train_data = ranges[:int(np.round(len(ranges)*0.6))]
        val_data=ranges[int(np.round(len(ranges)*0.6)):int(np.round(len(ranges)*0.8))]
        test_data = ranges[int(np.round(len(ranges)*0.8)):]
    else:
        train_data, val_data, test_data = [],[],[]

        for i in range(0,data_range,32):
            for j in range(0,data_range,32):
                for k in range(0,data_range,32):
                    idx = (i,j,k)
                    if i <=416 and j<= 416:
                        val_data.append(idx)
                    elif i>=484 and j>= 448 and k>= 448:
                        test_data.append(idx)
                    else:
                        train_data.append(idx)
# #build dataloader
params = {'batch_size': batch_size,
      'shuffle': True,
      'num_workers':20}

training_set, validation_set = Dataset(train_data,cat = target_cat,reg = target_class, vel = vel), Dataset(val_data, cat = target_cat,reg = target_class, vel = vel, normalize = normalize)
testing_set= Dataset(test_data, cat= target_cat, reg = target_class, vel = vel, normalize = normalize)
training_generator = data.DataLoader(training_set, **params)
validation_generator = data.DataLoader(validation_set, **params)
testing_generator = data.DataLoader(testing_set, **params)

In [ ]:
class BasicConv3d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv3d, self).__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, bias=False, **kwargs)

    def forward(self, x):
        x = self.conv(x)
        return F.relu(x, inplace=True)


# class InceptionE(nn.Module):

#     def __init__(self, in_channels, conv1_out, conv3_out, conv5_out):
#         super(InceptionE, self).__init__()
#         self.pool_out = 3
#         self.conv1_out = conv1_out
#         self.conv3_out = conv3_out
#         self.conv5_out = conv5_out
#         self.branch1x1 = BasicConv3d(in_channels, self.conv1_out, kernel_size=1)

#         self.branch3x3 = BasicConv3d(in_channels, self.conv3_out, kernel_size=3, padding = 1)
        

#         self.branch5x5 = BasicConv3d(in_channels, self.conv5_out, kernel_size=5, padding = 2)
#         self.branch_pool = BasicConv3d(in_channels, self.pool_out, kernel_size=1)


#     def forward(self, x):
#         branch1x1 = self.branch1x1(x)

#         branch3x3 = self.branch3x3(x)
        
#         branch5x5 = self.branch5x5(x)
        
#         branch_pool = F.avg_pool3d(x, kernel_size=3, stride=1, padding=1)
#         branch_pool = self.branch_pool(branch_pool)
#         outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
#         return torch.cat(outputs, 1)

class InceptionE2(nn.Module):

    def __init__(self, in_channels, conv1_out, conv3_out, conv5_out):
        super(InceptionE, self).__init__()
        self.pool_out = 3
        self.conv1_out = conv1_out
        self.conv3_out = conv3_out
        self.conv5_out = conv5_out
        self.branch1x1 = BasicConv3d(in_channels, self.conv1_out, kernel_size=1)

        self.branch3x3 = BasicConv3d(in_channels, self.conv3_out, kernel_size=3, padding = 1)
        

        self.branch5x5 = BasicConv3d(in_channels, self.conv5_out, kernel_size=5, padding = 2)
        self.branch_pool = BasicConv3d(in_channels, self.pool_out, kernel_size=1)


    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3(x)
        
        branch5x5 = self.branch5x5(x)
        
        branch_pool = F.avg_pool3d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        return torch.cat(outputs, 1)

class Inception_2(nn.Module):
    def __init__(self, channels, conv1_out, conv3_out, conv5_out, reg = 0):
        super(Inception, self).__init__()
        self.conv1_out = conv1_out
        self.conv3_out = conv3_out
        self.conv5_out = conv5_out
        self.incep = InceptionE2(channels, conv1_out = conv1_out, conv3_out = conv3_out, conv5_out = conv5_out)
        conv_in = conv1_out + conv3_out + conv5_out + 3
        self.conv1 = BasicConv3d(conv_in, conv_in, kernel_size = 3, padding = 1)
        self.conv2 = BasicConv3d(conv_in, conv_in//2, kernel_size = 3, padding = 1)
        self.reg = reg
        if self.reg:
            dim_out = 1
        else:
            dim_out = 2
        self.conv3 = BasicConv3d(conv_in//2, dim_out, kernel_size = 1)
    def forward(self, x):
        b_size = x.size(0)
        incep1 = self.incep(x)
        #print('incep1 = ', incep1.size())
        conv1=self.conv1(incep1)
        conv2=self.conv2(conv1) 
        conv3=self.conv3(conv2)
        if self.reg:
            conv3 = conv3.squeeze(1)
        return conv3

In [12]:
test_input = torch.Tensor(1,1,32,32,32).to(device).float()
dim_in, dim_out = 1,1
model = Inception(dim_in, conv1_out, conv3_out, conv5_out).to(device)
model(test_input)

tensor([[[[[2.0549e+33, 4.1266e+33,        nan,  ..., 0.0000e+00,
            9.0964e+33, 8.1735e+33],
           [0.0000e+00, 1.0570e+33,        nan,  ..., 1.8465e+33,
            1.6574e+33, 4.2253e+33],
           [0.0000e+00, 3.2265e+33,        nan,  ..., 2.0583e+34,
            0.0000e+00, 3.6383e+33],
           ...,
           [0.0000e+00, 1.1814e+32, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 7.8195e+33],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.2876e+34,
            0.0000e+00, 1.4426e+33],
           [2.8130e+29, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 3.1188e+32]],

          [[5.9284e+32, 0.0000e+00,        nan,  ..., 6.3863e+33,
            7.6127e+33, 9.1942e+33],
           [6.5188e+33, 0.0000e+00,        nan,  ..., 1.2498e+34,
            0.0000e+00, 0.0000e+00],
           [8.5949e+33, 4.7134e+33,        nan,  ..., 1.8076e+34,
            0.0000e+00, 2.8901e+33],
           ...,
           [0.0000e+00, 9.1834e+31, 0.0

In [ ]:



# #set up device

# #build model
dim_out = 1
if vel == 1:
    dim_in = 4
else:
    dim_in = 1
    dim = 1 ## need to be changed later
if model_idx == 0:
    model = SimpleUnet(dim, target_class).to(device)
elif model_idx == 1:
    model = Baseline(dim, dim).to(device)
elif model_idx == 2:
    model = Inception(dim_in, conv1_out, conv3_out, conv5_out).to(device)
elif model_idx == 3:
    model = R2Unet(dim_in, dim_out, t = 3, reg = target_class).to(device)
elif model_idx == 4:
    mask_model = one_layer_conv(dim,one_layer_outchannel = 8,kernel_size = 3,non_linearity = 'ReLU6', transformation = 'sqrt_root'
                                , power = 0.25).to(device)
    #state_dict = torch.load('../trained_model/epoch_10_MSE.pth')
    state_dict = torch.load('./pretrained/' + C_model + '.pth')
    mask_model.load_state_dict('state_dict')
    pred_model = R2Unet(dim,dim,t=3,reg = target_class).to(device)
    model = two_phase_conv(mask_model,pred_model,thres = thres)
elif model_idx == 5:
    mask_model = R2Unet(dim_in, dim_out, t = 3).to(device)
    state_dict = torch.load('./pretrained/' + C_model + '.pth')
    mask_model.load_state_dict(state_dict)
    pred_model = R2Unet(dim_in,dim_out,t=3,reg = target_class).to(device)
    model = two_phase_conv(mask_model,pred_model)
elif model_idx == 6:
    model = R2Unet_atten(dim, dim, t = 3, reg = 0).to(device)

elif model_idx == 7:
    mask_model = Inception(dim_in, conv1_out, conv3_out, conv5_out).to(device)
    state_dict = torch.load('./pretrained/' + C_model + '.pth')

    mask_model.load_state_dict(state_dict)
    pred_model = R2Unet(dim_in,dim_out,t=3,reg = target_class).to(device)
    model = two_phase_conv(mask_model,pred_model)
elif model_idx == 8:
    model = Inception(dim, conv1_out, conv3_out, conv5_out, reg = target_class).to(device)
else:
    print('model not exist')

if load_model:
    model = torch.load('pretrained/mytraining.pt')
#criterion = nn.MSELoss().to(device) #yueqiu
#weight = torch.Tensor([0.99,0.05,0.05])
if target_class == 0:
    criterion = nn.CrossEntropyLoss(weight = get_loss_weight(loss_weight, num_class = 2)).to(device)
    print('criterion classification')
    #criterion = yfloss(weight = get_loss_weight(loss_weight, num_class = 2).to(device), w = yfloss_weight, device = device)
else:
    criterion = weighted_nn_loss(loss_weight)
    #criterion = nn.MSELoss() #yueqiu


optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=weight_decay)
initial_loss(training_generator, validation_generator, model, criterion, target_class)

for epoch in range(epochs):
    adjust_learning_rate(lr, optimizer, epoch)
    train(training_generator, model, criterion, optimizer, epoch, print_freq, target_class = target_class)
    #evaluate on validation set
    validate(validation_generator, model, criterion, epoch, target_class = target_class, save_name = save_name)
if len(plot_label) == 0:
    plot_label = '_' + str(target_class) + '_' + str(model_idx) + '_'
train_plot(TRAIN_LOSS,VAL_LOSS, VAL_ACC, VAL_RECALL, VAL_PRECISION, target_class, plot_label = plot_label)
if target_class == 0:
    if record_results:
        args = parse_args()
        f= open("all_results","a+")
        f.write("arguments: %s" %(args) + '\n')
        f.write('Test Loss {BEST_VAL_LOSS:.5f},  Test Accuracy {BEST_ACC:.4f},  Test Recall {BEST_RECALL:.4f},  \
        Precision {BEST_PRECISION:.4f}   F1 score  {BEST_F1SCORE:.4f}\n'.format( \
                    BEST_VAL_LOSS=BEST_VAL_LOSS,BEST_ACC=BEST_ACC, BEST_RECALL = BEST_RECALL, BEST_PRECISION = BEST_PRECISION, BEST_F1SCORE =  BEST_F1SCORE))
        f.close() 

